# File manager by Multi-thread processing

## Import necessary libraries:

In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from concurrent.futures import ThreadPoolExecutor

# Custom library
import script.ImageProcessing as IP

In [ ]:
# Avoid out of memory errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

## Load your trained Keras model:

Load the Keras model that you have trained for image classification.

In [ ]:
DIR = "DIRECTORY HERE" # Folder containing unorganized images
H5_FILE = "H5 KERAS MODEL HERE" # H5 file
MODEL = load_model(H5_FILE)  # Load your trained Keras model
IMAGE_SIZE = MODEL.layers[0].input_shape[0][1] 

## Collect image paths and create threads & Define a function to classify images and move them:

- The `organize_images` traverse through the directory containing the images you want to classify and organize.
- For each image, create a thread that will run the `classify_and_move` function in order to move the image to the corresponding folder based on the classification result. 

In [ ]:
def organize_images(input_folder, model, image_size):
    image_paths = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder)]
    
    progress_bar = tf.keras.utils.Progbar(len(image_paths), 
                                              width=20, 
                                              interval=0.2, 
                                              unit_name='image')
    
    def classify_and_move(image_path, model, image_size):
        image = IP.image_resize_square(image_path, image_size, aspect_ratio=True, padding=True)
        image = np.expand_dims(image.numpy(), axis=0)
        
        prediction = model.predict(image)
        predicted_class = np.argmax(prediction)

        class_name = f"class_{predicted_class}"
        destination_folder = os.path.join(DIR, class_name)
        os.makedirs(destination_folder, exist_ok=True)

        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(destination_folder, image_filename)
        shutil.move(image_path, new_image_path)
        progress_bar.add(1)
    
    max_threads = os.cpu_count() // 2 # Half number of concurrent threads
    with ThreadPoolExecutor(max_threads) as executor:
        for image_path in image_paths:
            executor.submit(classify_and_move, image_path, model, image_size)

## Execute

In [ ]:
organize_images(DIR, MODEL, IMAGE_SIZE)